<a href="https://colab.research.google.com/github/Sahar-DataScience/sentiment-analysis-Tunisian-reviews/blob/main/Hackathon_code_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [212]:
import pandas as pd

import numpy as np



In [213]:
data =pd.read_csv('/content/drive/MyDrive/Train.csv')

In [214]:
data.head()

,ID,content,score
0,65479,Revoyez vos prix ainsi que la lenteur ke prend...,0
1,84256,تجربة ممتازة وشيقة فعلا تعاملاتهم محترمة ماعدا...,1
2,31581,دفو على نصابين,-1
3,58926,J arriver à temps et malheureusement je me re...,1
4,17945,Je suis contente des services rendu par Jumia....,1


In [215]:
data.describe()

,ID,score
count,59408.000000,59408.000000
mean,65141.616668,0.382777
std,44264.865162,0.881385
min,0.000000,-1.000000
25%,21946.750000,-1.000000
50%,65856.500000,1.000000
75%,106198.750000,1.000000
max,142743.000000,1.000000


In [216]:
data.content[100:120]

100                          Y a pas darticles a acheter
101                                            goooooode
102    محلاها تسهل عليك كل ما يخص اورنج تسوف انترنات ...
103    Cest retard livraison et jattends livraison à ...
104                                    cest un peu mieux
105    نجمة واحدة حتى تصلحو غلطتكم معايا ولا انتم تغش...
106                         Cette application et parfait
107              Super jaime cette application trop bien
108    Beaucoup d artickes, a des prix cassés J atten...
109    التطبيق سيئ جدا جدا ميستحقش أنه يا خد نجمة واح...
110                                         No ses mover
111    يوم الاثنين طلبت من المتجر حاسوب ووصلني التأكي...
112                                            tros bien
113    Jai commandé une tablette pour enfant pendant ...
114                                            Cest Naze
115                                      Toujours propre
116                                ca me faclite baucoup
117                   راءع جدا 

In [217]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59408 entries, 0 to 59407
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       59408 non-null  int64 
 1   content  59408 non-null  object
 2   score    59408 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.4+ MB


# data cleaning

### List of characters to remove

In [218]:
 
chars_to_remove = [',','$','+','?','!','.','à']
# List of column names to clean
cols_to_clean = ['content']

# Loop for each column
for col in cols_to_clean:
    # Replace each character with an empty string
    for char in chars_to_remove:
        data[col] = data[col].str.replace(char, '')
        
    # Convert col to numeric
    #data_train[col] = pd.to_numeric(apps[col]) 

In [219]:
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [220]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

stop words union

In [221]:
stopwords_AR = set(stopwords.words('arabic'))
stopwords_ENG=set(stopwords.words('english'))
stopwords_FR= set(stopwords.words('french'))

In [222]:
my_stop_word_list = stopwords.words('arabic') + stopwords.words('english') + stopwords.words('french')

clean msg function

In [223]:
def clean_message(message):   

  
  message = message.lower()
  message = message.split()
  stemmer = PorterStemmer()
  message = [stemmer.stem(word) for word in message if word not in set(my_stop_word_list)]
  message = " ".join(message)
  return message

inputs construction

In [224]:
content_training = []
for i in range(0, len(data)):
    msg = clean_message(data.content[i])
    content_training.append(msg)

In [225]:
#from sklearn.feature_extraction.text import CountVectorizer
#cv = CountVectorizer(max_features=3000)
#X = cv.fit_transform(content_training).toarray()

In [226]:
X=content_training

In [227]:
Y= np.array(data["score"])

In [228]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [229]:
import numpy as np

#un=np.ones(6601)
#zero=np.zeros(6601)
#unminus=un*-1
#un
#zero
#unminus

In [230]:
#p=len(content_test)


In [231]:
#label_test=np.ones(p)
#for i in range(int(p/3)): 
  #label_test[i]=label_test[i]*0
#for i in range(int(p/3),int(2*p/3)):  
  #label_test[i]=label_test[i]*-1
#print(label_test)



len(label_test)=19803, label_test[6601*2]=1


### modele d'entrainnement

In [232]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

In [233]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(x_train)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(x_test)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [234]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [235]:
# Model Definition with CNN
model_cnn = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model_cnn.summary()
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 116, 128)          10368     
_________________________________________________________________
global_average_pooling1d_3 ( (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 178,689
Trainable params: 178,689
Non-trainable params: 0
_________________________________________________________________


In [236]:
num_epochs = 10 # train with 50
history_model_cnn = model_cnn.fit(padded, y_train, epochs=num_epochs, validation_data=(testing_padded, y_test))

Epoch 1/10
1486/1486 [==============================] - 19s 12ms/step - loss: -137.6413 - accuracy: 0.2952 - val_loss: -3545.2812 - val_accuracy: 0.4166
Epoch 2/10
1486/1486 [==============================] - 18s 12ms/step - loss: -10539.8827 - accuracy: 0.4195 - val_loss: -46633.7852 - val_accuracy: 0.3664
Epoch 3/10
1486/1486 [==============================] - 18s 12ms/step - loss: -83500.0315 - accuracy: 0.4029 - val_loss: -183369.2031 - val_accuracy: 0.4197
Epoch 4/10
1486/1486 [==============================] - 18s 12ms/step - loss: -273055.5122 - accuracy: 0.4034 - val_loss: -470365.1250 - val_accuracy: 0.4179
Epoch 5/10
1486/1486 [==============================] - 18s 12ms/step - loss: -701355.8241 - accuracy: 0.4059 - val_loss: -974329.1250 - val_accuracy: 0.4212
Epoch 6/10
1486/1486 [==============================] - 18s 12ms/step - loss: -1312262.2569 - accuracy: 0.4034 - val_loss: -1740952.2500 - val_accuracy: 0.4212
Epoch 7/10
1486/1486 [==============================] - 18

prediction 

In [237]:
test=pd.read_csv('/content/drive/MyDrive/Test.csv')

In [238]:
content_test = []
for i in range(0, len(test)):
    msg = clean_message(test.content[i])
    content_test.append(msg)

In [239]:
testing_seq = tokenizer.texts_to_sequences(content_test)
test_X_seq = pad_sequences(testing_seq,maxlen=max_length)

In [240]:
Ypredict = model_cnn.predict_classes(test_X_seq, verbose=1)

 34/619 [>.............................] - ETA: 1s

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


619/619 [==============================] - 2s 3ms/step


In [241]:
#accuracy1 = model.evaluate(test_X_seq, y_test)

In [242]:
#from sklearn.feature_extraction.text import CountVectorizer
#cv = CountVectorizer(max_features=3000)
#Y1 = cv.fit_transform(content_test).toarray()

In [245]:
submission = pd.DataFrame()
submission['ID'] = test.ID
submission['score'] = Ypredict
submission.to_csv('hackSubmissionCNNSaharBM3.csv', index=False)


In [244]:
#output=model_cnn.predict(T)